In [0]:
#Connection with Google Drive

#from google.colab import drive
#drive.mount('/content/gdrive')
#!cp -rv /content/train_processed.zip /content/gdrive/My\ Drive/train_processed.zip
#!rm /content/gdrive/My\ Drive/kaggle_data/train_processed.zip
#!cp -rv /content/gdrive/My\ Drive/train_processed.zip /content/gdrive/My\ Drive/kaggle_data/train_processed.zip

# Connecting with Kaggle and Storing Data locally

In [0]:
# Get data from Kaggle
!pip install kaggle
!mkdir ~/.kaggle
!mkdir /content/.kaggle
!ls ~/.kaggle

import json
token = {"username":"mielgosez", "key":"46ef90eb86d527d54aca090759ccbef9"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!chmod 600 /root/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

print("1. Downloading datasets")
!kaggle competitions download -c histopathologic-cancer-detection -p /content/

print("2.  Moving train set to train folder")
!mkdir /content/train
!unzip -qq /content/train.zip -d /content/train
print("3. Moving test set to test folder")
!mkdir /content/test
!unzip -qq /content/test.zip -d /content/test


print("4. Moving files to their corresponding folders")
!mkdir /content/train/0
!mkdir /content/train/1
import pandas as pd
import shutil
!unzip -qq /content/train_labels.csv.zip
file_assign = pd.read_csv("/content/train_labels.csv") 
file_assign = file_assign.values
for item in file_assign:
  if item[1] == 0:
    shutil.move("/content/train/"+item[0]+".tif", 
                "/content/train/0/"+item[0]+".tif")
  else:
    shutil.move("/content/train/"+item[0]+".tif", 
                "/content/train/1/"+item[0]+".tif")

- path is now set to: {/content}
1. Downloading datasets
  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 43.6MB/s]
 98% 5.00M/5.10M [00:00<00:00, 10.5MB/s]
100% 5.10M/5.10M [00:00<00:00, 10.4MB/s]
 99% 1.29G/1.30G [00:28<00:00, 46.5MB/s]
100% 1.30G/1.30G [00:28<00:00, 49.4MB/s]
100% 4.98G/4.98G [02:01<00:00, 48.6MB/s]
100% 4.98G/4.98G [02:01<00:00, 43.8MB/s]
2.  Moving train set to train folder
3. Moving test set to test folder
4. Moving files to their corresponding folders


In [0]:
# Verify that the 
import os
path, dirs, files = next(os.walk("/content/train/0"))
file_count = len(files)
print("Number of files in train folder cat 0:"+ str(file_count))

path, dirs, files = next(os.walk("/content/train/1"))
file_count = len(files)
print("Number of files in train folder cat 1:"+ str(file_count))

Number of files in train folder cat 0:130908
Number of files in train folder cat 1:89117


# Creating the Sequential Model

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/train"
test_dir = "/content/test"

# Parameters
image_size = 96
batch_size = 32


# Data Setting

general_datagen = ImageDataGenerator(
        preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.2)

test_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x)

# Aquí dejar la mariquera y poner todo boolean
train_generator = general_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='binary',
        subset='training')

validation_generator = general_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='binary',
        subset='validation')

Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [0]:
from PIL import Image
import pandas as pd

file_assign = pd.read_csv("/content/train_labels.csv") 
file_assign = file_assign.values

image_sizes = []
for item in file_assign:
  if item[1] == 0:
    im = Image.open("/content/test/"+item[0]+".tif")
  else:
    im = Image.open("/content/train/1/"+item[0]+".tif")
  image_sizes.append(im.size)

In [0]:
ps = pd.Series([tuple(i) for i in image_sizes])
counts = ps.value_counts()
print(counts)

(96, 96)    220025
dtype: int64


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.5

# Use sigmoid and binary cross-entropy to address the problem in a more accurate fashion.
# Another trick is to increase the kernel size to track 'bigger' details
# Batch normalization is:
# 
model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (image_size, image_size, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(1, activation = "sigmoid"))

# Compile the model
model.compile(Adam(0.01), loss = "binary_crossentropy", metrics=["accuracy"])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 92, 92, 32)        9216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 92, 92, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 92, 92, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

# 176021 and 44004 are the sizes provided in the bloc where train_generator
# and validation_generator were defined.
train_steps = np.ceil(176021 / batch_size)
val_steps = np.ceil(44004 / batch_size)

earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1)
history = model.fit_generator(train_generator, steps_per_epoch=train_steps, 
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    epochs=6,
                    callbacks=[reducel, earlystopper])

Epoch 1/6
5501/5501 [==============================] - 2252s 409ms/step - loss: 0.3116 - acc: 0.8726 - val_loss: 0.2668 - val_acc: 0.8905
Epoch 2/6
5501/5501 [==============================] - 2144s 390ms/step - loss: 0.2695 - acc: 0.8934 - val_loss: 0.2593 - val_acc: 0.8929
Epoch 3/6
5501/5501 [==============================] - 2133s 388ms/step - loss: 0.2480 - acc: 0.9045 - val_loss: 0.2805 - val_acc: 0.8881

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
Epoch 4/6
5501/5501 [==============================] - 2153s 391ms/step - loss: 0.2071 - acc: 0.9221 - val_loss: 0.2529 - val_acc: 0.9041
Epoch 5/6
5501/5501 [==============================] - 2150s 391ms/step - loss: 0.1986 - acc: 0.9250 - val_loss: 0.2153 - val_acc: 0.9170
Epoch 6/6
5501/5501 [==============================] - 2145s 390ms/step - loss: 0.1938 - acc: 0.9273 - val_loss: 0.2056 - val_acc: 0.9212


In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [14]:
!ls /content/test/ | wc -l
!unzip -qq /content/test.zip -d /content/test/test
test_generator = test_datagen.flow_from_directory(
        "/content/test/",
        target_size=(image_size, image_size),
        color_mode="rgb",
        batch_size=1,
        class_mode=None,
        shuffle=False)
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples, verbose=1)

57458
Found 57458 images belonging to 1 classes.


# Generating the report on Test data

In [30]:
from glob import glob
import os
from skimage.io import imread

base_test_dir = '/content/test/test/'
test_files = glob(os.path.join(base_test_dir,'*.tif'))
submission = pd.DataFrame()
file_batch = 5000
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print(max_idx)
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[4].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = (K_test - K_test.mean()) / K_test.std()
    predictions = model.predict(K_test)
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[["id", "label"]]])
submission.head()


57458
Indexes: 0 - 5000
57458
Indexes: 5000 - 10000
57458
Indexes: 10000 - 15000
57458
Indexes: 15000 - 20000
57458
Indexes: 20000 - 25000
57458
Indexes: 25000 - 30000
57458
Indexes: 30000 - 35000
57458
Indexes: 35000 - 40000
57458
Indexes: 40000 - 45000
57458
Indexes: 45000 - 50000
57458
Indexes: 50000 - 55000
57458
Indexes: 55000 - 60000


,id,label
0,b8c53202df10b76794c1daa23e8997e0ac9a6f94,0.237784
1,3a4c4b99fbff710accb62d5fa480d131e57845a5,0.053820
2,a98ab9e399ef57531448592c725bb6141dfb83e6,0.050331
3,45732e8cfd8d42b1fd1f97feaf748d3ba3e36474,0.013040
4,4eefeedfcc99914ee4ecb32bedfa93c19884d702,0.007161


Results are submitted to Kaggle

In [31]:
submission.to_csv("results.csv",index=False)
!kaggle competitions submit -c histopathologic-cancer-detection -f results.csv -m "2nd attempt"

100% 2.87M/2.87M [00:07<00:00, 407kB/s]
Successfully submitted to Histopathologic Cancer Detection

Accuracy: 0.9044